In [1]:
import numpy as np
import pandas as pd

In [2]:
def num_to_chair(arr):
    x = str(arr[0]+1)
    y = arr[1]
    y = chr(y+65)
    return x+y

In [3]:
def chair_to_num(arr):
    x = int(arr[:-1])-1
    y = arr[-1]
    y = ord(y)-65
    return (x, y)

## Start

In [4]:
df = pd.read_excel('BD SEAT .xlsx')
df.shape

(55600, 10)

In [5]:
filtered_df = df[(df['DepartureDate'] == '2022-06-26') & (df['FlightNumber'] == 56261)]
filtered_df = filtered_df.sort_values(['SeatBookDate', 'SeatBookDateTime', 'BookingBookDate'])

In [6]:
filtered_df['AssignedSeat'] = ''
filtered_df

,RecordLocator,PassengerID,UnitDesignator,BookingBookDate,DepartureDate,FlightNumber,DepartureStation,ArrivalStation,SeatBookDate,SeatBookDateTime,AssignedSeat
52617,TFH8TK,89212360,8A,2022-01-14 04:44:51.337,2022-06-26,56261,BOG,ADZ,2022-01-13,23:44:51.3366667,
52618,DYHS9T,89212859,8B,2022-01-14 04:54:24.353,2022-06-26,56261,BOG,ADZ,2022-01-13,23:54:24.3533333,
52316,ABDUXT,91092690,30A,2022-02-12 02:57:47.843,2022-06-26,56261,BOG,ADZ,2022-02-11,21:57:47.8433333,
52317,ABDUXT,91092686,30B,2022-02-12 02:57:47.843,2022-06-26,56261,BOG,ADZ,2022-02-11,21:57:47.8433333,
52579,TFFUPK,91097595,31A,2022-02-12 03:57:39.797,2022-06-26,56261,BOG,ADZ,2022-02-11,22:57:39.7966667,
...,...,...,...,...,...,...,...,...,...,...,...
51445,S7PSUF,99132047,16C,2022-06-02 14:11:00.680,2022-06-26,56261,BOG,ADZ,NaT,NaN,
52251,CGPMFR,99224909,20B,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,
52597,CGPMFR,99224908,4E,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,
51420,NB3KWJ,99925865,12B,2022-06-13 19:15:17.557,2022-06-26,56261,BOG,ADZ,NaT,NaN,


In [7]:
cost = np.zeros((32, 6))

In [8]:
cost[0,:] = 39
cost[0,[1,-2]] = 34
cost[1:5,:] = 34
cost[1:5,[1,-2]] = 29
cost[5:11,:] = 27
cost[5:11,[1,-2]] = 22
cost[11:13,:] = 29
cost[11:13,[1,-2]] = 24
cost[13:23,:] = 18
cost[13:23,[1,-2]] = 12
cost[23:,:] = 14
cost[23:,[1,-2]] = 9
# cost[-1,[1,-2]] = 9*100000000000000000000

In [9]:
# Asigna los que compraron silla
chairs = list(filtered_df[~filtered_df['SeatBookDate'].isna()]['UnitDesignator'])
passengers = list(filtered_df[~filtered_df['SeatBookDate'].isna()]['PassengerID'])
for i in range(len(chairs)):
    chair = chairs[i]
    passenger = passengers[i]
    cost[(chair_to_num(chair))] = passenger
    filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair

In [10]:
bookings = list(filtered_df[filtered_df['SeatBookDateTime'].isna()]['RecordLocator'])
booking_set = set(bookings)
total_cost = 0
for booking in booking_set:
    passengers = filtered_df.loc[(filtered_df['SeatBookDateTime'].isna()) & (filtered_df['RecordLocator'] == booking), 'PassengerID']
    chairs = list()
    for passenger in passengers:
        min_cost = cost.min()
        position = (np.asarray(cost == min_cost).nonzero()[0][0], np.asarray(cost == min_cost).nonzero()[1][0])
        last_position = (np.asarray(cost == min_cost).nonzero()[0][-1], np.asarray(cost == min_cost).nonzero()[1][-1])
        aux = chairs.copy()
        aux.append(last_position)
        # aux = list(aux)
        aux = np.array(aux)
        if len(chairs) == 0:
            chair = num_to_chair(position)
            total_cost += cost[(chair_to_num(chair))]
            cost[(chair_to_num(chair))] = passenger
            filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
            chairs.append(position)
        elif np.linalg.norm(aux, ord=-2) > 1.5:
            chair = num_to_chair(last_position)
            total_cost += cost[(chair_to_num(chair))]
            cost[(chair_to_num(chair))] = passenger
            filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
            chairs.append(last_position)
        elif np.asarray(cost == min_cost).nonzero()[0].shape[0] > 2:
            for i in range(np.asarray(cost == min_cost).nonzero()[0].shape[0]):
                test_position = (np.asarray(cost == min_cost).nonzero()[0][i], np.asarray(cost == min_cost).nonzero()[1][i])
                aux = chairs
                aux.append(test_position)
                aux = list(aux)
                aux = np.array(aux)
                if np.linalg.norm(aux, ord=-2) > 1.5:
                    chair = num_to_chair(test_position)
                    total_cost += cost[(chair_to_num(chair))]
                    cost[(chair_to_num(chair))] = passenger
                    filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                    chairs.append(test_position)
                    break
        else:
            min_cost = cost[cost > min_cost].min()
            position = (np.asarray(cost == min_cost).nonzero()[0][0], np.asarray(cost == min_cost).nonzero()[1][0])
            last_position = (np.asarray(cost == min_cost).nonzero()[0][-1], np.asarray(cost == min_cost).nonzero()[1][-1])
            aux = chairs
            aux.append(test_position)
            aux = list(aux)
            aux = np.array(aux)
            if np.linalg.norm(aux, ord=-2) > 2:
                chair = num_to_chair(last_position)
                total_cost += cost[(chair_to_num(chair))]
                cost[(chair_to_num(chair))] = passenger
                filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                chairs.append(last_position)
            elif np.asarray(cost == min_cost).nonzero()[0].shape[0] > 2:
                for i in range(np.asarray(cost == min_cost).nonzero()[0].shape[0]):
                    test_position = (np.asarray(cost == min_cost).nonzero()[0][i], np.asarray(cost == min_cost).nonzero()[1][i])
                    aux = chairs
                    aux.append(test_position)
                    aux = list(aux)
                    aux = np.array(aux)
                    if np.linalg.norm(aux, ord=-2) > 2:
                        chair = num_to_chair(test_position)
                        total_cost += cost[(chair_to_num(chair))]
                        cost[(chair_to_num(chair))] = passenger
                        filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                        chairs.append(test_position)
                        break

In [11]:
total_cost

2521.0

In [12]:
filtered_df[filtered_df['SeatBookDate'].isna()]

,RecordLocator,PassengerID,UnitDesignator,BookingBookDate,DepartureDate,FlightNumber,DepartureStation,ArrivalStation,SeatBookDate,SeatBookDateTime,AssignedSeat
51421,U62PXY,89080215,12C,2022-01-12 17:17:33.007,2022-06-26,56261,BOG,ADZ,NaT,NaN,10C
51424,U62PXY,89080216,12F,2022-01-12 17:17:33.007,2022-06-26,56261,BOG,ADZ,NaT,NaN,13F
51428,U62PXY,89080213,13D,2022-01-12 17:17:33.007,2022-06-26,56261,BOG,ADZ,NaT,NaN,10D
51429,U62PXY,89080218,13E,2022-01-12 17:17:33.007,2022-06-26,56261,BOG,ADZ,NaT,NaN,13D
52245,U62PXY,89080217,1B,2022-01-12 17:17:33.007,2022-06-26,56261,BOG,ADZ,NaT,NaN,13C
...,...,...,...,...,...,...,...,...,...,...,...
51445,S7PSUF,99132047,16C,2022-06-02 14:11:00.680,2022-06-26,56261,BOG,ADZ,NaT,NaN,16E
52251,CGPMFR,99224909,20B,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,29E
52597,CGPMFR,99224908,4E,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,21B
51420,NB3KWJ,99925865,12B,2022-06-13 19:15:17.557,2022-06-26,56261,BOG,ADZ,NaT,NaN,15E


In [19]:
min_cost = cost.min()
position = (np.asarray(cost == min_cost).nonzero()[0][-1], np.asarray(cost == min_cost).nonzero()[1][-1])
num_to_chair(position)

'4D'

In [13]:
filtered_df.to_excel("output.xlsx")  

In [14]:
filtered_df

,RecordLocator,PassengerID,UnitDesignator,BookingBookDate,DepartureDate,FlightNumber,DepartureStation,ArrivalStation,SeatBookDate,SeatBookDateTime,AssignedSeat
52617,TFH8TK,89212360,8A,2022-01-14 04:44:51.337,2022-06-26,56261,BOG,ADZ,2022-01-13,23:44:51.3366667,8A
52618,DYHS9T,89212859,8B,2022-01-14 04:54:24.353,2022-06-26,56261,BOG,ADZ,2022-01-13,23:54:24.3533333,8B
52316,ABDUXT,91092690,30A,2022-02-12 02:57:47.843,2022-06-26,56261,BOG,ADZ,2022-02-11,21:57:47.8433333,30A
52317,ABDUXT,91092686,30B,2022-02-12 02:57:47.843,2022-06-26,56261,BOG,ADZ,2022-02-11,21:57:47.8433333,30B
52579,TFFUPK,91097595,31A,2022-02-12 03:57:39.797,2022-06-26,56261,BOG,ADZ,2022-02-11,22:57:39.7966667,31A
...,...,...,...,...,...,...,...,...,...,...,...
51445,S7PSUF,99132047,16C,2022-06-02 14:11:00.680,2022-06-26,56261,BOG,ADZ,NaT,NaN,16E
52251,CGPMFR,99224909,20B,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,29E
52597,CGPMFR,99224908,4E,2022-06-03 17:22:31.197,2022-06-26,56261,BOG,ADZ,NaT,NaN,21B
51420,NB3KWJ,99925865,12B,2022-06-13 19:15:17.557,2022-06-26,56261,BOG,ADZ,NaT,NaN,15E
